# 勾配ブースティング

各分類木（弱学習器）の予測誤差（＝損失の勾配）を、次の分類木で学習していく。
葉に相当する分類木の最大の深さはだいたい３から６である。 （ AdaBoostは深さ1の木が多い。）
AdaBoostとは対照的に、勾配ブースティングは予測誤差を訓練データの重みの割り当てに使うのではなく、次の決定木を訓練するための目的変数の設定に直接使う。さらに、決定木ごとに重みの項を使うのではなく、全体で1つの学習率（全て同じ値）を使う。

勾配ブースティングはsklearn.ensemble.GradientBoostingClassifierとして実装されているが、勾配ブースティングが逐次的なプロセスであり、その訓練に時間がかかる可能性がある。XGBoostの方が人気。他にもLightGBMやCatBoostなぢ、いくつかの勾配ブースティング実装が人気を集めている。  
scikit-learnはLightGBMにヒントを得て、GradientBoostingClassifierよりも高性能なHistGradientBoostingClassifierを実装している。  
序盤の木と終盤の木の重みが同じなのが違和感。

[参考サイト](https://zenn.dev/dalab/articles/9c843f0ec8aabf)  
ただ、2本目以降の決定木の目的変数は$y-\Sigma \hat{y}$とすべきだと思う。

In [1]:
import pandas as pd

df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)

df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

# drop 1 class
df_wine = df_wine[df_wine['Class label'] != 1]

y = df_wine['Class label'].values
X = df_wine[['Alcohol', 'Hue']].values

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test =\
            train_test_split(X, y, 
                             test_size=0.40, 
                             random_state=1)

In [3]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.01,
                          max_depth=4,random_state=1, use_label_encoder=False)
gbm = model.fit(X_train, y_train)
y_train_pred = gbm.predict(X_train)
y_test_pred = gbm.predict(X_test)
gbm_train = accuracy_score(y_train, y_train_pred)
gbm_test = accuracy_score(y_test, y_test_pred)
print(f'XGBoost train/test accuracies: {gbm_train:.3f}/{gbm_test:.3f}')

/Users/nagairyousuke/機械学習Python/StudyML/StudyML/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [19:06:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost train/test accuracies: 0.986/0.896


## アルゴリズム
もちろんです！
ここでは、\*\*勾配ブースティング（Gradient Boosting）\*\*のアルゴリズムを「ステップごとに」「分類と回帰の両方に通じる一般形」でわかりやすく解説します。

---

## ✅ 勾配ブースティングの概要

**目的：**
複数の「弱学習器（通常は決定木）」を**逐次的に組み合わせて**、最終的に高精度な予測モデルを構築する。

**コアアイデア：**
「前回の予測の誤り（損失関数の勾配）を説明するように、次の学習器を学習させる」

---

## 🔁 アルゴリズム（一般形）

与えられた：

* 入力データ $\{(x_1, y_1), ..., (x_N, y_N)\}$
* 損失関数 $L(y, F(x))$（例：平方誤差、ロジスティック損失など）
* 初期モデル $F_0(x)$
* 学習率（learning rate） $\eta \in (0,1]$
* イテレーション数 $M$

---

### 📌 ステップごとの流れ：

---

### Step 1. **初期化**

$$
F_0(x) = \arg\min_{\gamma} \sum_{i=1}^{N} L(y_i, \gamma)
$$

* 例：回帰（平方誤差）の場合、$F_0(x) = \text{平均}(y_i)$

---

### Step 2. **Boosting を繰り返す（m = 1 to M）**

#### Step 2-1. 勾配（または残差）を計算：

各データ点 $i$ について：

$$
r_{im} = -\left[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right]_{F(x_i) = F_{m-1}(x_i)}
$$

* これは「**今のモデルでどのくらいズレてるか**」の指標（勾配 or 残差）

---

#### Step 2-2. 残差 $r_{im}$ をターゲットとして、回帰木 $h_m(x)$ を学習：
現在のモデルの損失関数に対する負の勾配（residual）を近似するように学習器 を訓練する。
**損失関数を最も減らす方向（＝負の勾配方向）を目的変数**として、次の決定木を学習。

$$
x_i \mapsto r_{im}
\quad \text{で学習し、} \quad h_m(x) = \sum_{j=1}^{J_m} \gamma_{jm} \cdot \mathbf{1}_{x \in R_{jm}}
$$

* データを $J_m$ 個のリージョン（葉）$R_{jm}$ に分割
* 各葉での出力値 $\gamma_{jm}$ を求める

---

#### Step 2-3. 各リージョン $R_{jm}$ ごとに最適な補正量 $\gamma_{jm}$ を決定：

$$
\gamma_{jm} = \arg\min_{\gamma} \sum_{x_i \in R_{jm}} L(y_i, F_{m-1}(x_i) + \gamma)
$$

$\gamma_{jm}$の最適化だけで、勾配が表には出てこないが、しっかり勾配と2階導関数と同義となっている。  
単なる勾配を使って更新するより厳密。
* この補正量は「このリージョンに属する点を、どれくらい補正すれば損失が最小になるか」

---

#### Step 2-4. モデルを更新：

$$
F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)
$$

* $\eta$ は学習率。値が小さいほど学習は遅いが安定。

---

### Step 3. 最終モデル出力：

$$
F_M(x) = F_0(x) + \eta \sum_{m=1}^{M} h_m(x)
$$

---

### 📊 具体例（回帰の場合）

* 損失関数 $L(y, \hat{y}) = \frac{1}{2}(y - \hat{y})^2$
* 勾配は $r_i = y_i - F_{m-1}(x_i)$（＝残差）
* 木はこの残差を予測するように学習され、補正量 $\gamma_{jm}$ は単純な平均

---

### ✅ ポイントまとめ

| 項目         | 内容                        |
| ---------- | ------------------------- |
| モデル構築方法    | 決定木を逐次的に追加                |
| 学習目標       | 前回までの誤差（勾配）を修正            |
| 学習率 $\eta$ | 過学習を防ぎながら更新するための係数        |
| 出力値（葉の値）   | 損失を最小にする補正量 $\gamma_{jm}$ |

### 勘違い
木が連なるわけではない。各決定木は独立。  
誤差が小さくなるようにどんどん木を作っていく。どこの分岐になるかによって加算される値は変わる。  
序盤の木と終盤の木の重みが同じなのが違和感。

### ✅ なぜすべて同じ重みで足すのか？

これは勾配ブースティングの基本的な設計思想に基づいています：

* 各ステップでは「損失関数の勾配方向」に沿って少しだけ修正する（＝微調整する）
* その「ステップ幅」を一定にすることで、**過学習を防ぎつつ安定した収束**を目指す
* この「ステップ幅」に相当するのが **学習率（learning rate, η）**

---

### ❗ でも、1番目とK番目の木を等しく扱うのは問題？

確かに、現実には：

* **初期の木は大きな誤差を修正**し、
* **後半の木は微調整をしている**だけ

なので、同じ重みで扱うのが最適とは限りません。

---

### 🛠 そこで登場するのが「重み付け」や「正則化」

以下のような改良版がよく使われます：

#### ① **Shrinkage（縮小）**

* 各木の出力に学習率 $\eta$ を掛けて「弱く」する
* すべて同じ η だが、小さな η にして**たくさんの木で調整する**

#### ② **Weighted Boosting**

* 各ステップで木に異なる重み（たとえばデータ分割に応じた γ）を導入する方法もあり
* 例：XGBoostでは、葉の出力に正則化項を含めて最適な出力値を計算している

#### ③ **早期終了（early stopping）**

* 木の数を増やしすぎると過学習になるので、途中で打ち切る（Kが大きすぎないようにする）

---

### 🔍 実装例：XGBoostやLightGBM

* XGBoostでは、各木の出力（葉ごとの値）を最小化された目的関数に基づいて決定
* このとき、正則化項を入れて「重要でない木の影響を小さくする」

---

### 🧠 まとめ

| 観点               | 説明                                               |
| ---------------- | ------------------------------------------------ |
| 全ての木に同じ重みを与えるのは？ | ✔️ 基本形としてはOK                                     |
| 現実には？            | ❗ 最適でない場合も多い                                     |
| 改良策              | 学習率（η）の調整、正則化、early stopping、XGBoostのような高度な重み最適化 |

---

必要なら、XGBoostの目的関数の導出や、葉ごとの出力の最適化についても数式込みで説明できます！
